# Import President data from XML file

## Standard imports

In [1]:
# %load imports.py
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
import seaborn as sns
sns.set()
import lxml.etree as ET
from datetime import date


## Create empty dataframe with column and index names

In [2]:
df = pd.DataFrame(columns='first_name last_name dob dod bplace bstate tstart tend party'.split(), index=pd.Index(data=[], name='term')) 
df

,first_name,last_name,dob,dod,bplace,bstate,tstart,tend,party
term,,,,,,,,,


## Month lookup
A dictionary to translate the month name (e.g., 'Jan') into the integer month number

In [3]:
months = 'Jan Feb Mar Apr May Jun Jul Aug Sep Oct Nov Dec'.split()
month_lookup = { month:num for num, month in enumerate(months, 1)}
month_lookup

{'Jan': 1,
 'Feb': 2,
 'Mar': 3,
 'Apr': 4,
 'May': 5,
 'Jun': 6,
 'Jul': 7,
 'Aug': 8,
 'Sep': 9,
 'Oct': 10,
 'Nov': 11,
 'Dec': 12}

## Function to find date values and make a Python date object
We use xpath (`.//TAG`) to find the date tag, then grab the text from the sub tags. The year and day must be converted to integers, and the month value is a string
that is looked up in the `month_lookup` dictionary.

In [4]:
def get_date(doc, tag):
    year = row.findtext(f'.//{tag}/year')
    if year == '':  # still alive
        return date.today()
    month = row.findtext(f'.//{tag}/month')
    day = row.findtext(f'.//{tag}/day')

    return date(int(year), month_lookup[month], int(day))

In [5]:
doc = ET.parse('../DATA/presidents.xml')
for row in doc.findall('.//president'):
    term = row.findtext('index')
    first_name = row.findtext('.//name/first')
    last_name = row.findtext('.//name/last')
    dob = get_date(doc, 'birth')
    dod = get_date(doc, 'death')
    bplace  = row.findtext('birthplace')
    bstate = row.findtext('birthstate')
    tstart = get_date(doc, 'termstart')
    tend = get_date(doc, 'termstart')
    party = row.findtext('party')
    df.loc[term] = {   # add row to dataframe
        "first_name": first_name, "last_name": last_name, "dob": dob, "dod": dod,
        'bplace': bplace, 'bstate': bstate, 'tstart': tstart, 'tend': tend, 'party': party,
    }

df.tail()

,first_name,last_name,dob,dod,bplace,bstate,tstart,tend,party
term,,,,,,,,,
42,William Jefferson ''Bill'',Clinton,1946-08-19,2023-10-13,Hope,Arkansas,1993-01-20,1993-01-20,Democratic
43,George Walker,Bush,1946-07-06,2023-10-13,New Haven,Connecticut,2001-01-20,2001-01-20,Republican
44,Barack Hussein,Obama,1961-08-04,2023-10-13,Honolulu,Hawaii,2009-01-20,2009-01-20,Democratic
45,Donald John,Trump,1946-06-14,2023-10-13,"Queens, NYC",New York,2017-01-20,2017-01-20,Republican
46,Joseph Robinette,Biden,1942-11-10,2023-10-13,Scranton,Pennsylvania,2021-01-20,2021-01-20,Democratic


## Find first-born president for each party

Use `groupby()` to group presidents by party, then use `idxmin()` to find the index of the first entry with the minimum value. 
Then we look up the row with that index, which returns a Series, and index into the series with the column names.

In [6]:
for party, party_data in df.groupby('party'):
    min_index = party_data['dob'].idxmin() 
    president = df.loc[min_index]
    print(party, president.first_name, president.last_name, president.dob)

Democratic Andrew Jackson 1767-03-15
Democratic - Republican Thomas Jefferson 1743-04-13
Federalist John Adams 1735-10-30
Republican Andrew Johnson 1808-12-29
Whig William Henry Harrison 1773-02-09
no party George Washington 1732-02-22


## Find youngest president at start of term
It's easy to add another column to the dataframe with the age at the start of their first term by subtracting the date of birth from the term start date. That returns a `timedelta` object representing the elapsed time. That object has a `days` attribute, so we divide the days by 365.25 to get the approximate number of years. 

In [14]:
df['term_age'] = [round(d.days / 365.25, 1) for d in (df['tstart'] - df['dob'])]
index_of_youngest = df.term_age.idxmin()
df.loc[index_of_youngest]

first_name         Theodore
last_name         Roosevelt
dob              1858-10-27
dod              1919-01-06
bplace        New York City
bstate             New York
tstart           1901-09-14
tend             1901-09-14
party            Republican
term_age               42.9
Name: 26, dtype: object

## Count presidents in each party
The `value_counts()` method will count distinct occurences of values.

In [12]:
df.value_counts('party')

party
Republican                 20
Democratic                 16
Democratic - Republican     4
Whig                        4
Federalist                  1
no party                    1
Name: count, dtype: int64

## Count presidents from each state


In [13]:
df.value_counts('bstate')

bstate
Virginia          8
Ohio              7
New York          5
Massachusetts     4
Vermont           2
Texas             2
Pennsylvania      2
North Carolina    2
New Jersey        2
New Hampshire     1
South Carolina    1
Arkansas          1
California        1
Missouri          1
Kentucky          1
Iowa              1
Illinois          1
Hawaii            1
Georgia           1
Connecticut       1
Nebraska          1
Name: count, dtype: int64